### Cu Estimator example

This notebook demonstrates work of the Copper Estimator.
Current version (v1) of the estimator based on the numerical solution of numerical solution of a system of equations. `least_squares` is used as a loss function. Please check [the docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html#scipy-optimize-least-squares) for more details.

**Important:**
Don't forget to install our application as a pip package. Use `pip install -e .` in the project root directory.

In [ ]:
import json
import os

from src.cu_estimator import CuEstimator
from src.yield_estimator import YieldEstimator
from src.data_loader import DataLoader
from definitions import PATH_DATA_DIRECTORY

In [ ]:
data_loader = DataLoader(os.path.join(PATH_DATA_DIRECTORY, '1'))
df = data_loader.df_previous_production

df.head()

---

We need to train the estimator on the dataset.

In [ ]:
cu_estimator = CuEstimator()
cu_estimator.fit(df)

yield_estimator = YieldEstimator()
yield_estimator.fit(df)

Now we can predict the amount of copper for the given recipe.

_In fact, you should never use the same data for train and evaluation=)_

In [ ]:
row = df.iloc[2]

predicted_prt = cu_estimator.predict(row)
delta = row.cu_pct - predicted_prt
print(f'cu_prt: {row.cu_pct}, predicted prt: {predicted_prt} (delta = {delta})')

---

### Interactive example

Here you can try out our estimators with your own input params 

In [ ]:
import ipywidgets as widgets
from IPython.display import display

In [ ]:
def make_form():
    bush = widgets.IntSlider(min=0, max=1000, value=300, step=10, description='Bushlling')
    pig_iron = widgets.IntSlider(min=0, max=1000, value=200, step=10, description='Pig Iron')
    shred = widgets.IntSlider(min=0, max=1000, value=300, step=10, description='Shred')
    skulls = widgets.IntSlider(min=0, max=1000, value=250, step=10, description='Skulls')
    grade = widgets.Dropdown(options=['ST1', 'ST2', 'ST3'], value='ST1', description='Steel grade')


    def btn_click(_):
        row = pd.Series({
            'steel_grade': grade.value,
            'bushling': bush.value,
            'pig_iron': pig_iron.value,
            'municipal_shred': shred.value,
            'skulls': skulls.value
        })
        print(f'ST: {grade.value}, '
              f'Bushling: {bush.value}, '
              f'Pig Iron: {pig_iron.value}, '
              f'Shred: {shred.value}, '
              f'Skulls: {skulls.value}. '
              f'Cu: {cu_estimator.predict(row):.4f}%, '
              f'Yield: {yield_estimator.predict(row):.1f}kg')


    btn = widgets.Button(description='Predict')
    btn.on_click(btn_click)

    box = widgets.VBox([bush, pig_iron, shred, skulls, grade, btn])
    display(box)

In [ ]:
make_form()